<a href="https://colab.research.google.com/github/MoP16/ml-july-2025-a-eportfolio/blob/main/main_forensics_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Setup, Path Correction, and Testing

from google.colab import drive
import sys
import os

try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(e)

project_path = '/content/drive/My Drive/91 - TEMPORARY/251013-Intelligent Agents Individual Project/'

if project_path not in sys.path:
    sys.path.append(project_path)

from agent.file_scanner import FileScanner

print("--- Testing FileScanner ---")

scan_target_path = os.path.join(project_path, 'evidence_to_scan')
file_types_to_find = ['*.docx', '*.pdf', '*.jpg']

scanner = FileScanner(scan_path=scan_target_path, file_filters=file_types_to_find)
list_of_found_files = scanner.scan_directory()

if list_of_found_files:
    print("\n--- Metadata for Found Files ---")
    for file in list_of_found_files:
        metadata = scanner.get_file_metadata(file)
        if metadata:
            print(f"  - Path: {os.path.basename(metadata['file_path'])}")
            print(f"    Size: {metadata['size_bytes']} bytes")
            print(f"    Modified: {metadata['last_modified']}")

In [ ]:
# Testing the Archiver module ---
import os
from agent.file_scanner import FileScanner
from agent.archiver import Archiver

# --- Configuration ---
scan_target_path = os.path.join(project_path, 'evidence_to_scan')
archive_output_path = os.path.join(project_path, 'output', 'archives')
file_types_to_find = ['*.docx', '*.pdf']

# --- Execution ---
print("--- Testing Archiver ---")

# Scanner to find files
scanner = FileScanner(scan_path=scan_target_path, file_filters=file_types_to_find)
list_of_found_files = scanner.scan_directory()

if list_of_found_files:
    # Initialize the archiver
    archiver = Archiver(archive_path=archive_output_path)

    # Hashing function
    print("\n--- Testing Hashing Function ---")
    first_file = list_of_found_files[0]
    file_hash = archiver.calculate_hash(first_file)
    print(f"SHA-256 for '{os.path.basename(first_file)}': {file_hash}")

    # Archive
    print("\n--- Testing Archiving Function ---")
    archive_file = archiver.create_archive(list_of_found_files)

    if archive_file and os.path.exists(archive_file):
        print(f"\nVerification successful: '{os.path.basename(archive_file)}' exists in the output folder.")
else:
    print("No files found to test the archiver.")

In [ ]:
# This cell tests the full find, preserve, and record workflow
import os
from agent.file_scanner import FileScanner
from agent.archiver import Archiver
from agent.logger import Logger

# Configuration ---
scan_target_path = os.path.join(project_path, 'evidence_to_scan')
archive_output_path = os.path.join(project_path, 'output', 'archives')
log_output_path = os.path.join(project_path, 'output', 'logs')
file_types_to_find = ['*.docx', '*.pdf']

# Execution ---
print("--- Testing Full Agent Workflow ---")

# Initialize all modules
logger = Logger(log_path=log_output_path)
scanner = FileScanner(scan_path=scan_target_path, file_filters=file_types_to_find)
archiver = Archiver(archive_path=archive_output_path)

# Operation
logger.log_action("AGENT_START", details=f"Scanning {scan_target_path}")

# Scan for files
list_of_found_files = scanner.scan_directory()

if list_of_found_files:
    # Log each found file and its hash
    for file in list_of_found_files:
        file_hash = archiver.calculate_hash(file)
        if file_hash:
            logger.log_action("EVIDENCE_FOUND", details=f"File: {os.path.basename(file)}, SHA256: {file_hash}")
        else:
            logger.log_action("HASH_FAILED", status="FAILURE", details=f"File: {os.path.basename(file)}")

    # Create the archive
    archive_file = archiver.create_archive(list_of_found_files)

    # Log the result of the archiving
    if archive_file:
        logger.log_action("ARCHIVE_CREATED", details=f"Archive: {os.path.basename(archive_file)}")
    else:
        logger.log_action("ARCHIVE_FAILED", status="FAILURE")
else:
    logger.log_action("SCAN_COMPLETE", details="No relevant files found.")

# Export the final log to a CSV for review
logger.export_logs_to_csv(log_output_path)

print("\n--- Workflow Test Finished ---")

In [ ]:
# --- This cell tests the FULL end-to-end agent workflow ---
import os
from agent.file_scanner import FileScanner
from agent.archiver import Archiver
from agent.logger import Logger
from agent.transmitter import Transmitter

# --- Configuration ---
scan_target_path = os.path.join(project_path, 'evidence_to_scan')
archive_output_path = os.path.join(project_path, 'output', 'archives')
log_output_path = os.path.join(project_path, 'output', 'logs')
remote_dest_path = os.path.join(project_path, 'remote_server_destination') # Our simulated server
file_types_to_find = ['*.docx', '*.pdf', '*.jpg']

# --- Execution ---
print("--- Starting Full Digital Forensics Agent Workflow ---")

# 1. Initialize all modules
logger = Logger(log_path=log_output_path)
scanner = FileScanner(scan_path=scan_target_path, file_filters=file_types_to_find)
archiver = Archiver(archive_path=archive_output_path)
transmitter = Transmitter(destination_url=remote_dest_path)

# 2. Log start
logger.log_action("AGENT_START", details=f"Scanning {scan_target_path}")

# 3. Scan for files
list_of_found_files = scanner.scan_directory()

if list_of_found_files:
    # 4. Log each found file and its hash
    for file in list_of_found_files:
        file_hash = archiver.calculate_hash(file)
        logger.log_action("EVIDENCE_FOUND", details=f"File: {os.path.basename(file)}, SHA256: {file_hash}")

    # 5. Create the archive
    archive_file = archiver.create_archive(list_of_found_files)

    if archive_file:
        logger.log_action("ARCHIVE_CREATED", details=f"Archive: {os.path.basename(archive_file)}")

        # 6. Transfer the archive
        transfer_success = transmitter.secure_transfer(archive_file)

        # 7. Log the transfer result
        if transfer_success:
            logger.log_action("TRANSFER_COMPLETE", details=f"Archive sent to {remote_dest_path}")
        else:
            logger.log_action("TRANSFER_FAILED", status="FAILURE")
    else:
        logger.log_action("ARCHIVE_FAILED", status="FAILURE")

# 8. Export the final log
logger.export_logs_to_csv(log_output_path)

print("\n--- Full Agent Workflow Finished ---")

In [ ]:
import os

# Define your project path
project_path = '/content/drive/My Drive/91 - TEMPORARY/251013-Intelligent Agents Individual Project/'

# Change the current working directory to your project path
try:
    os.chdir(project_path)
    print(f"Changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The path '{project_path}' was not found. Please check for typos.")

# Now, run the test discovery from the project root.
# It will automatically find the 'tests' subdirectory.
print("\n--- Running Unit Tests ---")
!python -m unittest discover tests